<a href="https://colab.research.google.com/github/krldlamini/edm_brazildengue/blob/main/Seasonal_Surrogates/notebooks/Surrogates_Rho_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The method of surrogates

This file runs ccm for all surrogate series generated in R, for each climatic variable and stores these in csv files


# Installing pyEDM

In [ ]:
!pip install pyEDM

# Importing packages

In [ ]:
from pyEDM import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# for memory management
#import os
#import psutil
import gc #to collect garbarge/ memory optimisation
from scipy.integrate import odeint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Uploading and fixing data for analysis

In [ ]:
file_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Combined_Files/355030_A701_combined.csv'
data = pd.read_csv(file_path, delimiter = ',')
data = data.drop(columns = ['data']) # drop dates column
data = data.dropna() # dropping NA's
data_norm = (data-data.mean())/data.std() # normalising the data

# Uploading surrogate data

In [ ]:
rain_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/Rain_Surrogates.csv'
tot_rain = pd.read_csv(rain_path, delimiter = ',')
max_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/MaxTemp_Surrogates.csv'
max_temp = pd.read_csv(max_path, delimiter = ',')
min_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/MinTemp_Surrogates.csv'
min_temp = pd.read_csv(min_path, delimiter = ',')

## Add Cases to the Surrogate data

Total Precipitation

In [ ]:
data_norm = data_norm.reset_index(drop=True)
first_two_columns = data_norm.iloc[:, :2]
tot_rain = tot_rain.reset_index(drop=True)
surr_tot_rain = pd.concat([first_two_columns, tot_rain], axis=1)

Maximum Temperature

In [ ]:
data_norm = data_norm.reset_index(drop=True)
first_two_columns = data_norm.iloc[:, :2]
max_temp = max_temp.reset_index(drop=True)
surr_max_temp = pd.concat([first_two_columns, max_temp], axis=1)

Minimum Temperature

In [ ]:
data_norm = data_norm.reset_index(drop=True)
first_two_columns = data_norm.iloc[:, :2]
min_temp = min_temp.reset_index(drop=True)
surr_min_temp = pd.concat([first_two_columns, min_temp], axis=1)

## Perform CCM for original time series and surrogate data

**Maximum temperature**

In [ ]:
CCM_maxtemp = CCM( dataFrame = data_norm, E = 4,
     columns = 'cases', target = 'temperatura_maxima_na_hora_ant',
     libSizes = '10 770 10', sample = 10, includeData=True);

In [ ]:
c_names = surr_tot_rain.columns.tolist()[2:] # get surrogate column names

In [ ]:
result_max = pd.DataFrame(observed_max)
for target in c_names:

    CCM_max_surr = CCM(dataFrame = surr_tot_max, E=4, columns='cases',
                            target= target, libSizes='10 770 10', sample=10)

    # Extract the library size and Y:X
    temp_max = CCM_max_surr.iloc[:, :2]

    # If result_max is empty, initialize it; otherwise, left merge on library size
    if result_max.empty:
        result_max = temp_max
    else:
        result_max = pd.merge(result_max, temp_max, on='LibSize', how='left')

    #save data to csv
    file_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/'
    pd.DataFrame(result_max).to_csv(file_path + 'MaxCCM_Surr.csv', sep =';', index=False)


**Minimum** **temperature**

In [ ]:
CCM_mintemp = CCM( dataFrame = data_norm, E = 4,
     columns = 'cases', target = 'temperatura_minima_na_hora_ant',
     libSizes = '10 770 10', sample = 10, includeData = True, showPlot = True)

In [ ]:
result_min = pd.DataFrame(observed_min)
for target in c_names:

    CCM_min_surr = CCM(dataFrame = surr_tot_min, E=4, columns='cases',
                            target= target, libSizes='10 770 10', sample=10)

    # Extract the library size and Y:X
    temp_min = CCM_min_surr.iloc[:, :2]

    # If result_min is empty, initialize it; otherwise, left merge on library size
    if result_min.empty:
        result_min = temp_min
    else:
        result_min = pd.merge(result_min, temp_min, on='LibSize', how='left')

    #save data to csv
    file_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/'
    pd.DataFrame(result_min).to_csv(file_path + 'MinCCM_Surr.csv', sep =';', index=False)

**Total** **Precipitation**

In [ ]:
CCM_precipitation = CCM( dataFrame = data_norm, E = 4,
     columns = 'cases', target = 'precipitacao_total_horario',
     libSizes = '10 770 10', sample = 10)


In [ ]:
result_rain = pd.DataFrame(observed_rain)
for target in c_names:

    CCM_rain_surr = CCM(dataFrame = surr_tot_rain, E=4, columns='cases',
                            target= target, libSizes='10 770 10', sample=10)

    # Extract the library size and Y:X
    temp_rain = CCM_rain_surr.iloc[:, :2]

    # If result_rain is empty, initialize it; otherwise, left merge on library size
    if result_rain.empty:
        result_rain = temp_rain
    else:
        result_rain = pd.merge(result_rain, temp_rain, on='LibSize', how='left')

    #save data to csv
    file_path = 'drive/MyDrive/Dengue_BR/Output_Documents/Surrogate_Data/'
    pd.DataFrame(result_rain).to_csv(file_path + 'RainCCM_Surr.csv', sep =';', index=False)
